Install ffmpeg and libavcodec-dev (plus related stuff) from
https://launchpad.net/~mc3man/+archive/ubuntu/trusty-media

Install OpenCV using the instructions from 
http://www.pyimagesearch.com/2015/06/22/install-opencv-3-0-and-python-2-7-on-ubuntu/

In my case I had a build error related to missing CUDA libs, so the build command I used was

    cmake -D CMAKE_BUILD_TYPE=RELEASE \
            -D CMAKE_INSTALL_PREFIX=/usr/local \
            -D INSTALL_C_EXAMPLES=OFF \
            -D INSTALL_PYTHON_EXAMPLES=ON \
            -D "OPENCV_EXTRA_MODULES_PATH=../../opencv_contrib/modules" \
            -D BUILD_EXAMPLES=ON \
            -D WITH_CUDA=OFF ..



In [1]:
print "OK"

OK


In [ ]:
!sudo pip install pytube

In [ ]:
# Reset GPU
#!sudo rmmod nvidia_uvm
#!sudo rmmod nvidia
#!sudo insmod /lib/modules/3.13.0-77-generic/kernel/drivers/video/nvidia.ko
#!sudo insmod /lib/modules/3.13.0-77-generic/kernel/drivers/video/nvidia-uvm.ko
#!echo "OK"

In [ ]:
!mkdir videos
!mkdir imageseries

In [ ]:
!ls videos

In [ ]:
from pytube import YouTube

# not necessary, just for demo purposes.
from pprint import pprint

yt = YouTube("https://www.youtube.com/watch?v=yim23IFJOyQ")

yt.set_filename('Chimpanzees1')
# Once set, you can see all the codec and quality options YouTube has made
# available for the perticular video by printing videos.

# pprint(yt.get_videos())

In [ ]:
#video = yt.get('mp4', '360p')

In [ ]:
# In this case, we'll need to specify both the codec (mp4) and resolution
# (either 360p or 720p).

# Okay, let's download it!
video = yt.get('mp4', '360p')
#video.download('./videos/')


In [ ]:
# Now let's try to read this using our fresh opencv build..

In [2]:
import cv2
import h5py


In [3]:
from IPython.display import HTML, FileLink, display
from base64 import b64encode
def link_video(path):
    src = FileLink(path)
    video_tag = '<video controls alt="test" src="%s">' % (path)
    
    display(HTML(data=video_tag))


In [4]:
vc = cv2.VideoCapture('videos/Chimpanzees1.mp4')
frames = 0
shape = None
dtype = None
while (True):
    ret, img = vc.read()
    # img should be an numpy ndarray of shape (width, heigth, channels)
    if (not ret):
        break
    if (shape is None):
        shape = img.shape
    if (dtype is None):
        dtype = img.dtype
    frames+=1
    
print "Could read %d frames of shape %r and dtype %r" % (frames, shape, dtype)

Could read 1490 frames of shape (360, 640, 3) and dtype dtype('uint8')


In [ ]:
#import h5py

#h5f = h5py.File('videos/Chimpanzees1_7.hdf5',driver='core', backing_store=True)
#h5f = h5py.File('/tmp/Chimpanzees1_7.hdf5')
#h5g = h5f.create_group('input')

In [5]:
import numpy as np

In [6]:
shape

(360, 640, 3)

In [7]:
rshape = [ shape[2],shape[0], shape[1] ]

In [8]:
ishape = tuple([ frames ] + list(rshape))
ichunkshape = tuple([ 1 ] + list(rshape))
imaxshape = tuple([ None ] + list(rshape))

In [9]:
import numpy as np
from numpy.random import permutation


In [10]:
#inframes = h5g.create_dataset('raw_hsl_frames', 
#                             shape=ishape, dtype=np.float32, 
#                             chunks=ichunkshape, maxshape=imaxshape,
#                             fletcher32=True, fillvalue=0)
#
inframes = np.zeros(shape=ishape, dtype=np.float32)

In [11]:
permuted_indices = permutation(range(frames))

In [12]:
permuted_indices

array([119, 797, 583, ..., 241,  39, 999])

In [13]:
vc = cv2.VideoCapture('videos/Chimpanzees1.mp4')
frames = 0
shape = None
dtype = None
while (True):
    ret, img = vc.read()
    if (not ret):
        break
    hsvimg = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    
    hsvimgf = hsvimg.astype(np.float32) / 255.0
    hsvimgf = hsvimgf.swapaxes(2,0).swapaxes(1,2)
    # img should be an numpy ndarray of shape (width, heigth, channels)
    inframes[permuted_indices[frames],:,:,:] = hsvimgf
    
    #hsvimg2 = (inframes[permuted_indices[frames],:,:,:] * 255.0).astype(np.uint8)
    
    #print np.sum(hsvimg-hsvimg2)
    #bgrimg = cv2.cvtColor(hsvimg2, cv2.COLOR_HSV2BGR)
    #print np.sum(img-bgrimg)
    
    #cv2.imwrite("videos/recoded1_%05d.png" % (frames), bgrimg)
    
    frames+=1
    if (frames % 300==0):
        print "Frame #%d" % (frames)
print "Read %d frames" % (frames)


Frame #300
Frame #600
Frame #900
Frame #1200
Read 1490 frames


In [ ]:
#inframesmem = np.array(inframes)

In [ ]:
#!rm videos/Chimpanzees1_recoded.mp4
#!ffmpeg -i imageseries/Chimpanzees1_%05d.png -c:v libx264 -b:v 400k -vf format=yuv420p -maxrate:v 500k -bufsize 1000k  -r 10 -preset medium videos/Chimpanzees1_recoded.mp4

In [ ]:
#link_video('videos/Chimpanzees1_recoded.mp4')

In [14]:
import os
import h5py
from glob import glob
import numpy as np


## MXNet ##



In [15]:
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')
logging.debug("test")

import os
# Comment this out, once it all works
#os.environ['MXNET_ENGINE_TYPE']='NaiveEngine'


06:38:46 DEBUG:test


In [16]:
logger = logging.getLogger()


In [17]:
import mxnet as mx
import mxnet.io as mxio


In [18]:
logging.debug("test")

06:38:58 DEBUG:test


In [19]:
a = mx.nd.ones((2,3), mx.cpu())

In [20]:
b = mx.nd.zeros((2,3), mx.cpu())

In [21]:
a[1:] = 2
a.copyto(b)

In [22]:
b.asnumpy()

array([[ 1.,  1.,  1.],
       [ 2.,  2.,  2.]], dtype=float32)

In [23]:
windowsize=128
batch_size=32

In [24]:
a = mx.nd.ones(shape=(2,2,2))
print a[1:2].shape

(1L, 2L, 2L)


In [25]:
class MultiplyMaskOp(mx.operator.CustomOp):
    
    def forward(self, is_train, req, in_data, out_data, aux):
        img_channels = in_data[1].asnumpy()
        mask = in_data[0].asnumpy()
        
        if req[0]=='null':
            return
        
        shp = list(img_channels.shape)
        shp[1] += 1
        
        out = np.zeros(shape=tuple(shp))
        out[:,0:1,:,:] = mask
        out[:,1:,:,:] = img_channels*mask # Automatic broadcasting to the rescue
        
        if req[0]=='write':
            out_data[0][:] = out
        elif req[0]=='add':
            out_data[0][:] += out
                
    def backward(self, req, out_grad, in_data, out_data, in_grad, aux):
        if req[0]=='null':
            return
        
        mask = in_data[0]
        img_channels = in_data[1]
        
        img_grad = out_data[:,1:]
        mask_grad = out_data[:,0:1]
        
        # Gradient of mask is copied through
        
        if req[0]=='write':
            in_grad[0][:] = mask_grad
        elif req[0]=='add':
            in_grad[0][:] += mask_grad
           
        img_grad = np.ones(img_channels.shape)
        for i in range(3):
            # Product rule applies for image channels
            chan_grad = mx.nd.add(mx.nd.multiply(img_channels[:,i:i+1], mask_grad), 
                                  mx.nd.multiply(img_grad[:,i:i+1], mask) )
            img_grad[:,i,:,:] = chan.asnumpy()
            
        if req[1]=='write':
            in_grad[1][:] = img_grad    
        elif req[1]=='add':
            in_grad[1][:] += img_grad
            
@mx.operator.register("maskmult")
class MultiplyMaskOpProp(mx.operator.CustomOpProp):
    
    def __init__(self):
        super(MultiplyMaskOpProp, self).__init__()
        
    def list_arguments(self):
        return ['mask', 'data']

    def list_outputs(self):
        return ['masked_data']

    def infer_shape(self, in_shape):
        mask_shape = list(in_shape[0])
        data_shape = list(in_shape[1])
        data_shape[1]+=1
        return in_shape, [data_shape], []
    
    def create_operator(self, ctx, shapes, dtypes):
        return MultiplyMaskOp()
    

In [26]:
data = mx.symbol.Variable('data')
y = mx.symbol.Variable('y')

net = data
dil = 1
for d in range(1,5):
    net = mx.symbol.Convolution(net, kernel=(1,1), num_filter=d*16+2, name='squeeze%d' % (d), workspace=200)
    net1 = mx.symbol.LeakyReLU(net, act_type='prelu')
    net = mx.symbol.Convolution(net1, kernel=(3,3), num_filter=32*d+2, dilate=(dil,dil), name='conv%d' % (d), workspace=200)
    net2 = mx.symbol.LeakyReLU(net, act_type='prelu')
    
    # We need to crop net1, since it can be larger than net2 
    net1 = mx.symbol.Crop(net1, net2, center_crop=True)  
    
    net = mx.symbol.Concat(net1,net2)
    #print net.infer_shape(data=(10,4,64,64))
    net = mx.symbol.DropoutAllChannels(net, p=0.05)
    dil *= 2

net = mx.symbol.Convolution(net, kernel=(1,1), num_filter=128, name='squeeze_final', workspace=200)
net = mx.symbol.Dropout(net, p=0.2)
net = mx.symbol.LeakyReLU(net, act_type='prelu')
   
net = mx.symbol.Convolution(net, kernel=(3,3), num_filter=3, name='decoder',  workspace=200)
net = mx.symbol.Activation(net, name='final_activation', act_type='sigmoid')
out = mx.symbol.MAERegressionOutput(data=net, label=y, name='output_')

def get_output_resolution(input_shape=(10, 3, 640, 480)):
    mshape = list(input_shape)
    mshape[1] = 1
    return net.infer_shape(data=input_shape, mask=mshape)[-2][0][2:]

def get_middle_slice(X):
    res = get_output_resolution(X.shape)
    dx = (X.shape[2]-res[0]) // 2
    dy = (X.shape[3]-res[1]) // 2
    return X[:,:,dx:X.shape[2]-dx,dy:X.shape[3]-dy]



In [31]:
num_epoch = 1
#model = mx.model.FeedForward(ctx=mx.cpu(), symbol=out, num_epoch=num_epoch, optimizer='sgd',
#                             learning_rate=0.05, momentum=0.9, wd=0.00001)

model = mx.model.FeedForward(ctx=[ mx.gpu(i) for i in range(1)], symbol=out, num_epoch=num_epoch, optimizer='adam')
print "OK"

OK


In [28]:
from numpy.lib.stride_tricks import as_strided

def tile_array(a, b1, b2):
    r, c = a.shape
    rs, cs = a.strides
    x = as_strided(a, (r, b1, c, b2), (rs, 0, cs, 0))
    return x.reshape(r*b1, c*b2)

def tile_batch_mask(shape, p, tile_x, tile_y):
    target = np.ones(shape=shape, dtype=np.float32)
    assert(shape[1]==1)
    for i in range(target.shape[0]):
        
        rnd_shape = (shape[2] / tile_x, shape[3] / tile_y)
        rnd = (np.random.random(rnd_shape)<=p).astype(np.float32)
        tiled = tile_array(rnd, tile_x, tile_y)
        target[i,0,:,:] = tiled
    return target


In [29]:
model.num_epoch = 1
model.load('dilated-autoencoder2_4', 2)

MXNetError: [18:41:21] src/io/local_filesys.cc:149: Check failed: allow_null  LocalFileSystem: fail to open "dilated-autoencoder2_4-symbol.json"

In [32]:

chunk_size = batch_size*16
for i in range(2):
    window_off_x =  np.random.randint(0, inframes.shape[2]-windowsize)
    window_off_y = np.random.randint(0, inframes.shape[3]-windowsize)
    start = np.random.randint(0, 1300-chunk_size)
    train_X = inframes[start:start+chunk_size,:,window_off_x:window_off_x+windowsize,window_off_y:window_off_y+windowsize]
    train_y = get_middle_slice(train_X)
    mask_shape = list(train_X.shape)
    mask_shape[1] = 1
    if (i%2==0):
        mask = tile_batch_mask(mask_shape, 0.80, 16, 16)
    else:
        mask = tile_batch_mask(mask_shape, 0.9, 32, 32)
    
    train_X = np.concatenate((mask, train_X*mask), axis=1)
    
    test_X = inframes[1300:,:,window_off_x:window_off_x+windowsize,window_off_y:window_off_y+windowsize]
    test_y = get_middle_slice(test_X)
    test_mask_shape = list(test_X.shape)
    test_mask_shape[1] = 1
    if (i%2==0):
        test_mask = tile_batch_mask(test_mask_shape, 0.80, 16, 16)
    else:
        test_mask = tile_batch_mask(test_mask_shape, 0.90, 32, 32)
    
    
    
    test_X = np.concatenate((test_mask, test_X*test_mask), axis=1)
    train_iter = mx.io.NDArrayIter({'data' : train_X}, { 'y' : train_y }, shuffle=True, batch_size=batch_size)
    test_iter = mx.io.NDArrayIter({'data' : test_X}, { 'y' : test_y}, shuffle=True, batch_size=batch_size)
    print "New Windowed Batch %i" % (i)
    model.fit(train_iter,
          eval_data=test_iter,
          eval_metric="mae",
          batch_end_callback=mx.callback.Speedometer(batch_size))
    if (i % 5==4):
        model.save('dilated-autoencoder2_%d' % (i))

06:42:49 INFO:Start training with [gpu(0)]


New Windowed Batch 0


06:43:10 INFO:Epoch[0] Resetting Data Iterator
06:43:10 INFO:Epoch[0] Time cost=20.413
06:43:13 INFO:Epoch[0] Validation-mae=0.150589
06:43:13 INFO:Start training with [gpu(0)]


New Windowed Batch 1


06:43:29 INFO:Epoch[0] Resetting Data Iterator
06:43:29 INFO:Epoch[0] Time cost=15.975
06:43:32 INFO:Epoch[0] Validation-mae=0.161410


In [ ]:
size = 0
for k in model.arg_params:
    size += np.prod(model.arg_params[k].shape)
print "Model size: ",size

In [ ]:
mx.viz.plot_network(net)

In [ ]:
out.list_arguments()

In [ ]:
arg_shape, out_shape, aux_shape = out.infer_shape(X=(10, 3, 640,480), y=(10,3,270,110))
dict(zip(net.list_arguments(), arg_shape))
size = 0
for s in arg_shape:
    print s
    size += np.prod(s)
print "Total parameter size=%d MB, total temp size=%d MB" % (size / (1024*1024), 640*480*1000*8 / (1024*1024))

In [ ]:
net.list_arguments()

#### Note


 * Important Paper about Dilated Convolution: http://arxiv.org/pdf/1511.07122v1.pdf

 * Deep Probabilistic Inference Models http://gitxiv.com/posts/oRw692PEooNcwh9Qh/neural-variational-inference-for-text-processing



## Keras Models ##


In [ ]:

import numpy as np
np.random.seed(1337)  # for reproducibility


from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape, Permute
from keras.layers.advanced_activations import ThresholdedReLU
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution2D, ZeroPadding2D
from keras.datasets import imdb
from keras.optimizers import SGD
from keras.regularizers import l1,l2,l1l2, ActivityRegularizer, activity_l1
from keras import backend as K
import keras.layers.convolutional as kconv
import tensorflow as tf
import tensorflow as tf
from keras.layers import MaskedLayer
import keras.backend as K
from keras.layers.advanced_activations import PReLU
from keras.layers.normalization import BatchNormalization

_FLOATX = K.floatx()


In [ ]:
print 'Build model...X'

num_feature_detectors=12*12*3
tile_size=12
receptive_field_size=16
model = Sequential()
model.add(Dropout(0.001, input_shape=ichunkshape[1:]))

input_features = Convolution2D( name='encode_tiled_features', 
                        nb_filter=num_feature_detectors, nb_row=receptive_field_size,nb_col=receptive_field_size, 
                        subsample=(tile_size,tile_size), 
                        border_mode='valid', trainable=True,
                        dim_ordering='tf' )


print np.prod(ichunkshape[1:])
model.add(input_features)

print input_features.output_shape 

model.add(Activation('relu'))

decoder = Convolution2D( name='decode_tiled_features', 
                        nb_filter=tile_size*tile_size*3, nb_row=1,nb_col=1, subsample=(1,1),
                        border_mode='same', trainable=True,
                        dim_ordering='tf')

model.add(decoder)
model.add(Reshape(tuple(list(decoder.output_shape[1:3]) + [tile_size, tile_size, 3])))
print model.output_shape
model.add(Permute((1,3,2,4,5)))

print model.output_shape
model.add(Reshape((model.output_shape[1]*model.output_shape[2],model.output_shape[3]*model.output_shape[4],model.output_shape[5])))

model.add(ZeroPadding2D(((ichunkshape[1]-model.output_shape[1])/2, (ichunkshape[2]-model.output_shape[2])/2), 
                        dim_ordering='tf'))

print model.output_shape
def mean_abs_error_all(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis=None)



print model.output_shape

In [ ]:
model.compile(optimizer='adam', loss=mean_abs_error_all, class_mode='binary')

In [ ]:
def data_generator(batchsize):
    for i in range(10):
        for j in permutation(1300):
            bi = inframesmem[j:(j+batchsize),:,:,:]

            yield bi,  bi

for i in range(1):
    print "Over-Epoch %i" % (i)
    model.fit_generator(data_generator(40), validation_data=(inframes[1400:1430], inframes[1400:1430]),
              show_accuracy=False, nb_epoch=1, samples_per_epoch=500, nb_worker=1)

model.save_weights('model1.hdf5')

In [ ]:
def data_generator(batchsize):
    for i in range(10):
        for j in permutation(1300):
            bi = inframesm[j:(j+batchsize),:,:,:]
            yield bi,  bi

for i in range(1):
    print "Over-Epoch %i" % (i)
    model.fit_generator(data_generator(4), validation_data=(inframes[1400:1430], inframes[1400:1430]),
              show_accuracy=False, nb_epoch=5, samples_per_epoch=400, nb_worker=1)

In [ ]:
print 'Build model...X'

num_feature_detectors=16*16*4
tile_size=16
receptive_field_size=24
model = Sequential()
model.add(Dropout(0.1, input_shape=ichunkshape[1:]))

input_features = Convolution2D( name='encode_tiled_features', 
                        nb_filter=num_feature_detectors, nb_row=receptive_field_size,nb_col=receptive_field_size, 
                        subsample=(tile_size,tile_size), 
                        border_mode='valid', trainable=True,
                        dim_ordering='tf', activity_regularizer=activity_l1(0.0000005))


model.add(input_features)
model.add(PReLU())

decoder = Convolution2D( name='decode_tiled_features', 
                        nb_filter=tile_size*tile_size*3, nb_row=1,nb_col=1, subsample=(1,1),
                        border_mode='same', trainable=True,
                        dim_ordering='tf')

model.add(decoder)
model.add(Reshape(tuple(list(decoder.output_shape[1:3]) + [tile_size, tile_size, 3])))
print model.output_shape
model.add(Permute((1,3,2,4,5)))

print model.output_shape
model.add(Reshape((model.output_shape[1]*model.output_shape[2],model.output_shape[3]*model.output_shape[4],model.output_shape[5])))

model.add(ZeroPadding2D(((ichunkshape[1]-model.output_shape[1])/2, (ichunkshape[2]-model.output_shape[2])/2), 
                        dim_ordering='tf'))

print model.output_shape
def mean_abs_error_all(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis=None)



print model.output_shape

In [ ]:
model.compile(optimizer='adam', loss=mean_abs_error_all, class_mode='binary')


In [ ]:
def data_generator(batchsize):
    for i in range(10):
        perm = permutation(1300)
        for i in range(len(perm)//batchsize):
            bi = inframesmem[perm[i*batchsize:(i+1)*batchsize],:,:,:]
            yield bi,  bi

for i in range(1):
    print "Over-Epoch %i" % (i)
    model.fit_generator(data_generator(30), validation_data=(inframes[1400:1430], inframes[1400:1430]),
              show_accuracy=False, nb_epoch=5, samples_per_epoch=400, nb_worker=1)

In [ ]:
print 'Build model...X'

num_feature_detectors=12*12*3
tile_size=12
receptive_field_size=16
model = Sequential()
model.add(Dropout(0.1, input_shape=ichunkshape[1:]))

enc1 = Convolution2D( name='enc1', 
                        nb_filter=num_feature_detectors, nb_row=receptive_field_size,nb_col=receptive_field_size, 
                        subsample=(tile_size,tile_size), 
                        border_mode='valid', trainable=True,
                        dim_ordering='tf', activity_regularizer=activity_l1(0.0000005))


model.add(enc1)
model.add(PReLU())
model.add(Dropout(0.1))
# ---------------
tile_size2=2
num_feature_detectors2=tile_size2*tile_size2*num_feature_detectors 
receptive_field_size2=2

enc2 = Convolution2D( name='enc2', 
                        nb_filter=num_feature_detectors2, nb_row=receptive_field_size2,nb_col=receptive_field_size2, 
                        subsample=(tile_size2,tile_size2), 
                        border_mode='valid', trainable=True,
                        dim_ordering='tf', activity_regularizer=activity_l1(0.0))


model.add(enc2)
model.add(PReLU())

# --------------------

dec2 = Convolution2D( name='dec2', 
                        nb_filter=tile_size2*tile_size2*num_feature_detectors, nb_row=1,nb_col=1, subsample=(1,1),
                        border_mode='valid', trainable=True,
                        dim_ordering='tf')

model.add(dec2)
# We need to reshape and reorder dimensions, so we can, at least in theory, do symmetric, layer-wise training...
newshape2 = tuple(list(dec2.output_shape[1:3])+ [tile_size2, tile_size2, num_feature_detectors])
model.add(Reshape(newshape2))
model.add(Permute((1,3,2,4,5)))
blockshape2 = (model.output_shape[1]*model.output_shape[2], model.output_shape[3]*model.output_shape[4], model.output_shape[5])

model.add(Reshape(blockshape2))

# --------------------

dec1 = Convolution2D( name='dec1', 
                        nb_filter=tile_size*tile_size*3, nb_row=1,nb_col=1, subsample=(1,1),
                        border_mode='same', trainable=True,
                        dim_ordering='tf')
model.add(dec1)
newshape1 = tuple(list(dec1.output_shape[1:3])+ [tile_size, tile_size, 3])
model.add(Reshape(newshape1))
model.add(Permute((1,3,2,4,5)))

blockshape1 = (model.output_shape[1]*model.output_shape[2],model.output_shape[3]*model.output_shape[4], model.output_shape[5])

model.add(Reshape(blockshape1))

model.add(ZeroPadding2D(((ichunkshape[1]-model.output_shape[1])/2, (ichunkshape[2]-model.output_shape[2])/2), 
                        dim_ordering='tf'))

# --------------------

print model.output_shape
def mean_abs_error_all(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis=None)



print model.output_shape

In [ ]:
model.compile(optimizer='adam', loss=mean_abs_error_all, class_mode='binary')

In [ ]:
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], T.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

In [ ]:
def data_generator(batchsize):
    while True:
        perm = permutation(1400)
        for i in range(len(perm)//batchsize):
            bi = inframesmem[perm[i*batchsize:(i+1)*batchsize],:,:,:]
            yield bi,  bi

for i in range(1):
    print "Over-Epoch %i" % (i)
    model.fit_generator(data_generator(40), validation_data=(inframes[1400:1480], inframes[1400:1480]),
              show_accuracy=False, nb_epoch=10, samples_per_epoch=500, nb_worker=1)

In [ ]:
model.save_weights('sparse_autoencoder_4layers.hdf5')


## Visualizing the reconstructed video ##

In [ ]:
import cv2

In [ ]:
!mkdir imageseries

In [ ]:
!rm imageseries/*.jpg

In [ ]:
frame.shape

In [ ]:
mask = tile_batch_mask((1,1,360, 640), 0.8, 4, 4)

In [ ]:
j = 0
c = 0

for i in permuted_indices:    
    #print np.sum(hsvimg-hsvimg2)
    fframe = inframes[i,:,:,:] # np.transpose(inframes[i,:,:,:], [1,2,0])
    fframe = fframe[np.newaxis,:,:,:]
    mfframe = np.concatenate((mask, fframe*mask), axis=1)
    bgi =  mx.io.NDArrayIter({'data' : mfframe}, batch_size=1)
    
    pframe = model.predict(bgi, num_batch=1)[0]
    pframe = np.clip(pframe, 0.0, 1.0, pframe)
    pframe = (pframe * 255.0).astype(np.uint8)
    piframe = np.transpose(pframe, [1,2,0])
    bgrimg = cv2.cvtColor(piframe, cv2.COLOR_HSV2BGR)

    j+=1
    c+=1
    cv2.imwrite("imageseries/reconstruction1_%05d.jpg" % (c), bgrimg)
    if (c>400):
        break
    if (j % 10==0):
        print "frame %d" % (c)
    
    

In [ ]:
from IPython.display import display, Image
display(Image('imageseries/reconstruction1_00002.jpg'))

In [ ]:
!rm imageseries/*.png

In [ ]:
!rm videos/Chimpanzees1_reconstructed3.mp4
!ffmpeg -i imageseries/reconstruction1_%05d.jpg -c:v libx264 -b:v 400k -maxrate:v 500k -bufsize 1000k -r 10 -preset medium videos/Chimpanzees1_reconstructed3.mp4


In [ ]:
from IPython.display import display, Image

In [ ]:
!ls -hs imageseries/recons*.jpg

In [ ]:
display(Image('imageseries/reconstruction1_00052.jpg'))

In [ ]:
link_video('videos/Chimpanzees1_reconstructed3.mp4')

In [ ]:
from IPython.display import FileLink
display(FileLink('videos/Chimpanzees1_reconstructed3.mp4'))

In [ ]:
link_video('videos/Chimpanzees1_reconstructed2.mp4')
#link_video('videos/Chimpanzees1_reconstructed2.mp4')

In [ ]:
link_video('videos/Chimpanzees1.mp4')

In [ ]:
display(FileLink('videos/Chimpanzees1_reconstructed2.mp4'))

In [ ]:
class array(object) :
    """Simple Array object that support autodiff."""
    def __init__(self, value, name=None):
        self.value = value
        if name:
            self.grad = lambda g : {name : g}

    def __add__(self, other):dilate
        assert isinstance(other, int)
        ret = array(self.value + other)
        ret.grad = lambda g : self.grad(g)
        return ret

    def __mul__(self, other):
        assert isinstance(other, array)
        ret = array(self.value * other.value)
        def grad(g):
            x = self.grad(g * other.value)
            x.update(other.grad(g * self.value))
            return x
        ret.grad = grad
        return ret

# some examples
a = array(1, 'a')
b = array(2, 'b')
c = b * a
d = c + 1
print d.value
print d.grad(1)

In [ ]:
(b*a).grad(2)